In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

### Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummary import summary
import copy
from datetime import time
import wandb

### Custom

In [ ]:
from cv_geoguessr.data.StreetViewImagesDataset import StreetViewImagesDataset

from cv_geoguessr.grid.grid_partitioning import Partitioning

from cv_geoguessr.utils.plot_images import plot_images

from cv_geoguessr.utils.evaluation import create_confusion_matrix


## Colab specific

Run only when using Colab.

In [ ]:
COLAB = False

if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

## Logging in via WandB

In [ ]:
wandb.init(project="CV-GeoGuessr", entity="cv-geoguessr")

## Model constants

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 100
CELL_WIDTH = 0.04

SAMPLES_TO_SHOW = 5

IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406], device=device)
IMAGENET_STD = torch.tensor([0.229, 0.224, 0.225], device=device)

CITY_BOUNDS_FILE = '../data/metadata/citybounds/london.csv'
CITY_BOUNDS_FILE_GCLOUD = '../data/metadata/city bounds/london.csv'

COLAB_LONDON_PHOTO_DIR = lambda train: f'/content/drive/MyDrive/Documents/University/2021-2022/CS4245 Computer Vision/data/images/{"train" if train else "test"}/london'
LOCAL_LONDON_PHOTO_DIR = lambda train: f'../data/images/london/{"train" if train else "test"}'
SJOERD_LONDON_PHOTO_DIR = lambda train: f'../data/images/{"train_img" if train else "test_img"}/london/'
GCLOUD_LONDON_PHOTO_DIR = lambda train: f'../data/images/{"train" if train else "test"}/'

LONDON_PHOTO_DIR = GCLOUD_LONDON_PHOTO_DIR

SESSION = "4th_test_start_from_second_20percent_data_augmentation"
BASE_FOLDER = "./checkpoints"
CHECKPOINT_FOLDER = f'{BASE_FOLDER}/{SESSION}/'

wandb.config.update({"train_batch_size": TRAIN_BATCH_SIZE, "test_batch_size": TEST_BATCH_SIZE, "cell_width": CELL_WIDTH})


### ImageNet setup

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt -O imagenet_classes.txt

In [ ]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

## Download ResNet50

In [ ]:
resnet50 = models.resnet50(pretrained=True, progress=True)
resnet50.to(device)

print('Downloaded ResNet50')

In [ ]:
summary(resnet50, (3, 224, 224))

## Load the data

### Set up grid portioning

In [ ]:
grid_partitioning = Partitioning(CITY_BOUNDS_FILE_GCLOUD, CELL_WIDTH)

grid_partitioning.plot()

### Create the data loaders

In [ ]:
# Add additional random transformation to augment the training dataset
data_transforms_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomPerspective(distortion_scale = .3, p = .4),
    transforms.Resize(256),
    transforms.CenterCrop((224, 224)),
    # transforms.RandomCrop(size = (224,224)),
    transforms.ColorJitter(brightness = 0.2, contrast = 0, saturation = 0.05, hue = 0.1),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

data_transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop((224, 224)),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

train_data_set = StreetViewImagesDataset(LONDON_PHOTO_DIR(True), grid_partitioning, data_transforms_train)
train_loader = DataLoader(train_data_set, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_data_set = StreetViewImagesDataset(LONDON_PHOTO_DIR(False), grid_partitioning, data_transforms_test)
test_loader = DataLoader(test_data_set, batch_size=TEST_BATCH_SIZE, shuffle=True)

data_loaders = {
    "train": train_loader,
    "val": test_loader
}


data_set_sizes = {
    'train':len(train_data_set),
    'val':len(test_data_set),
}

print(data_set_sizes)

### Preview some training images

In [ ]:
examples = enumerate(train_loader)
batch_idx, (eval_images, eval_coordinates) = next(examples)
eval_images = eval_images.to(device)
eval_coordinates = eval_coordinates.to(device)

plot_images(eval_images[:SAMPLES_TO_SHOW].cpu(), IMAGENET_MEAN.cpu(), IMAGENET_STD.cpu())

In [ ]:
eval_coordinates[0, :]

## Evaluate the model

In [ ]:
resnet50.eval()

check_images = eval_images[:SAMPLES_TO_SHOW].to(device)

with torch.no_grad():
    output = torch.nn.functional.softmax(resnet50(check_images), dim=1)

In [ ]:
for i in output:
    top5_prob, top5_catid = torch.topk(i, 5)

    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

    print()

In [ ]:
plot_images(check_images.to('cpu').cpu(), IMAGENET_MEAN.cpu(), IMAGENET_STD.cpu())

## Train on the grid output


In [ ]:
lr = 0.001
momentum = 0.9
gamma = 0.1
lr_decay_step = 7
num_epochs = 50

wandb.config.update({"lr": lr, "momentum": momentum, "gamma": gamma, "lr_decay_step": lr_decay_step, "epochs": num_epochs})

In [ ]:
from torch.optim import lr_scheduler
from torch import optim

number_of_grid_elements = len(grid_partitioning.cells)

wandb.config.update({"number_of_grid_elements": number_of_grid_elements})

for param in resnet50.parameters():
    param.requires_grad = False

resnet50.fc = nn.Linear(resnet50.fc.in_features, number_of_grid_elements)
resnet50.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# We might not want this for the initial few epochs I (sjoerd) think but lets just roll with it
optimizer_ft = optim.SGD(resnet50.fc.parameters(), lr=lr, momentum=momentum)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=lr_decay_step, gamma=gamma)

In [ ]:
summary(resnet50, (3, 224, 224))


In [ ]:
import os

if not os.path.isdir(CHECKPOINT_FOLDER):
    os.makedirs(CHECKPOINT_FOLDER)

torch.save(resnet50.state_dict(), CHECKPOINT_FOLDER + "0.ckpt")

In [ ]:
import copy
import time


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    """
    Trains a model, based on https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

    :param model: the model to train
    :param criterion: the criterion to use
    :param optimizer: the optimizer to use
    :param scheduler: torch.optim.lr_scheduler
    :param num_epochs:
    :return: a trained model
    """

    since = time.time()

    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        train_acc = 0
        test_acc = 0

        distance_error = {}
        distance_error_count = {}

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward: track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)

                    # Add distance error metric
                    _, actual_label_index = torch.max(labels, 1)
                    for index, label in enumerate(actual_label_index[preds != actual_label_index].tolist()):
                        distance_error.setdefault(label, 0)
                        distance_error[label] += (grid_partitioning.cells[label].centroid).distance(grid_partitioning.cells[preds[index]].centroid)

                        distance_error_count.setdefault(label, 0)
                        distance_error_count[label] += 1

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                _, actual_label_index = torch.max(labels, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == actual_label_index)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / data_set_sizes[phase]
            epoch_acc = running_corrects.double() / data_set_sizes[phase]
            
            # Append avg distance error
            avg_distance = 0
            for k in distance_error:
                avg_distance += distance_error[k] / distance_error_count[k]
            avg /= len(distance_error.keys())

            # writer.add_scalar(f"Loss/{phase}", epoch_loss, epoch)
            wandb.log({f"Loss/{phase}": epoch_loss, "epoch": epoch})
            # writer.add_scalar(f"Accuracy/{phase}", epoch_acc, epoch)
            wandb.log({f"Accuracy/{phase}": epoch_acc, "epoch": epoch})

            wandb.log({f"Distance/{phase}": avg, "epoch": epoch})

            if phase == 'train':
                train_acc = epoch_acc
            else:
                test_acc = epoch_acc

            print(f'{phase} loss: {epoch_loss:.4f} | accuracy: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())

        print(f"{train_acc}\t{test_acc}")

        torch.save(resnet50.state_dict(),
                   CHECKPOINT_FOLDER + f"epoch_{epoch}.ckpt")

    time_elapsed = time.time() - since

    print(
        f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model)

    return model

In [ ]:
# Lets train the model
resnet50 = train_model(resnet50, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
from collections import OrderedDict

def model_layers(model, input_size, batch_size=-1, device="cuda"):
    """Returns the layers of the model flattened, based on the summary code"""

    def register_hook(module):

        def hook(module, input, output):
            class_name = str(module.__class__).split(".")[-1].split("'")[0]
            module_idx = len(layers)

            m_key = "%s-%i" % (class_name, module_idx + 1)
            layers[m_key] = OrderedDict()
            layers[m_key]["input_shape"] = list(input[0].size())
            layers[m_key]["input_shape"][0] = batch_size
            if isinstance(output, (list, tuple)):
                layers[m_key]["output_shape"] = [
                    [-1] + list(o.size())[1:] for o in output
                ]
            else:
                layers[m_key]["output_shape"] = list(output.size())
                layers[m_key]["output_shape"][0] = batch_size

            params = 0
            layers[m_key]["params"] = module.parameters()
            if hasattr(module, "weight") and hasattr(module.weight, "size"):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                layers[m_key]["trainable"] = module.weight.requires_grad
                layers[m_key]["weights"] = module.weight
            if hasattr(module, "bias") and hasattr(module.bias, "size"):
                params += torch.prod(torch.LongTensor(list(module.bias.size())))
            layers[m_key]["nb_params"] = params

        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
            and not (module == model)
        ):
            hooks.append(module.register_forward_hook(hook))

    device = device.lower()

    assert device in [
        "cuda",
        "cpu",
    ], "Input device is not valid, please specify 'cuda' or 'cpu'"

    if device == "cuda" and torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # multiple inputs to the network
    if isinstance(input_size, tuple):
        input_size = [input_size]

    # batch_size of 2 for batchnorm
    x = [torch.rand(2, *in_size).type(dtype) for in_size in input_size]
    # print(type(x[0]))

    # create properties
    layers = OrderedDict()
    hooks = []

    # register hook
    model.apply(register_hook)

    # make a forward pass
    # print(x.shape)
    model(*x)

    # remove these hooks
    for h in hooks:
        h.remove()

    return layers


# load an old model
def load_model(model, PATH, lock_factor, device):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint)

    layers = model_layers(model, (3, 224, 224))
    n = len(layers)
    print(n)  # 10 layers
    # print(model_layers)

    for i, layer in enumerate(layers):
        # for layer_param in layer.parameters():
        #     layer_param.requires_grad = i > n * lock_factor
        for param in layers[layer]["params"]:
            param.requires_grad = i > n * lock_factor

    model.to(device)
    summary(model, (3, 224, 224))

    return model


In [ ]:
MODEL_TO_LOAD_PATH = BASE_FOLDER + "/second_test/epoch_24.ckpt"
lock_factor = 0.8

resnet50 = load_model(resnet50, MODEL_TO_LOAD_PATH, lock_factor, device)

In [ ]:
# now train with the loaded model with more layers unlocked

optimizer_ft = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)
resnet50 = train_model(resnet50, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
# Running the model evaluation
# def run_evaluation(model, partitions, predictions=True):
#     n = len(partitions.cells)
#     confusion_matrix = torch.zeros((n, n)).to(device)
#
#     i = 0
#     for inputs, labels in dataloaders['val']:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#
#         # forward
#         with torch.set_grad_enabled(False):
#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#
#         for label, output, pred in zip(labels, outputs, preds):
#             l = torch.argmax(label)
#             # print(l)
#
#             if predictions:
#                 confusion_matrix[l, :] += pred
#             else:
#                 confusion_matrix[l, :] += output
#
#             i += 1
#
#     return confusion_matrix


In [ ]:
confusion_matrix = create_confusion_matrix(resnet50, grid_partitioning, dataloaders['val'], False, device)
print(confusion_matrix.to('cpu'))

In [ ]:
from cv_geoguessr.utils.plot_results import plot_confusion_matrix

plot_confusion_matrix(confusion_matrix)